## Gibbs Sampler

In [10]:
# first, import the random package
import random


# Input:  Integers k, t, and N, followed by a collection of strings Dna
# Output: GibbsSampler(Dna, k, t, N)
def GibbsSampler(Dna, k, t, N):
    Motifs = RandomizedMotifSearch(Dna, k, t)
    BestMotifs=Motifs
    for j in range(N):
        i=random.randint(0,t-1)
        del Motifs[i]
        profile=ProfileWithPseudocounts(Motifs)
        kmer=ProfileGeneratedString(Dna[i], profile, k)
        Motifs.insert(i,kmer)
        if Score(Motifs) < Score(BestMotifs):
            BestMotifs=Motifs
    return BestMotifs

# place all subroutines needed for GibbsSampler below this line
def Pr(Text, Profile):
    p=1
    for i in range(len(Text)):
        p=p*Profile[Text[i]][i]
    return p

def Normalize(Probabilities):
    new_dict=Probabilities
    t=sum(new_dict.values())
    for i in new_dict:
        new_dict[i]/=t
    return new_dict

def WeightedDie(Probabilities):
    n = random.uniform(0, 1)
    for p in Probabilities:
        n -= Probabilities[p]
        if n <= 0:
            return p

def ProfileGeneratedString(Text, profile, k):
        n = len(Text)
        probabilities = {} 
        for i in range(0,n-k+1):
            probabilities[Text[i:i+k]] = Pr(Text[i:i+k], profile)
        probabilities = Normalize(probabilities)
        return WeightedDie(probabilities)
    
def RandomizedMotifSearch(Dna, k, t):
    M = RandomMotifs(Dna, k, t)
    BestMotifs = M
    while True:
        Profile = ProfileWithPseudocounts(M)
        M = Motifs(Profile,k,Dna)
        if Score(M) < Score(BestMotifs):
            BestMotifs = M
        else:
            return BestMotifs

def RandomMotifs(Dna, k, t):
    motifs=[]
    for i in range(t):
        num=random.randint(0,len(Dna[0])-k)
        motifs.append(Dna[i][num:num+k])
    return motifs

def Motifs(Profile,k,Dna):
    motifs=[]
    for i in range(len(Dna)):
        motifs.append(ProfileMostProbableKmer(Dna[i],k,Profile))
    return motifs

def ProfileWithPseudocounts(Motifs):
    t = len(Motifs)
    k = len(Motifs[0])
    #profile = {} # output variable
    profile=CountWithPseudocounts(Motifs)
    for key,v in profile.items():
        v[:]= [x/(t+4) for x in v]
    return profile

def CountWithPseudocounts(Motifs):
    k = len(Motifs[0])
    count = {'A':[1]*k,'C':[1]*k,'G':[1]*k,'T':[1]*k}
    t = len(Motifs)
    for i in range(t):
        for j in range(k):
            symbol = Motifs[i][j]
            count[symbol][j] += 1
    return count

def ProfileMostProbableKmer(text, k, profile):
    p=-1
    kmer=text[0:k]
    for i in range(len(text)-k+1):
        if Pr(text[i:i+k],profile)>p:
            p=Pr(text[i:i+k],profile)
            kmer=text[i:i+k]
    return kmer

def Score(Motifs):
    score=0
    t = len(Motifs)
    k = len(Motifs[0])
    consensus=Consensus(Motifs)
    for i in range(t):
        for j in range(k):
            if (Motifs[i][j]!=consensus[j]):
                score+=1
    return score

def Consensus(Motifs):
    k = len(Motifs[0])
    count = CountWithPseudocounts(Motifs)
    consensus = ""
    for j in range(k):
        m = 0
        frequentSymbol = ""
        for symbol in "ACGT":
            if count[symbol][j] > m:
                m = count[symbol][j]
                frequentSymbol = symbol
        consensus += frequentSymbol
    return consensus

In [11]:
Dna=["CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA","GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG","TAGTACCGAGACCGAAAGAAGTATACAGGCGT","TAGATCAAGTTTCAGGTGCACGTCGGTGAACC","AATCCACCAGCTCCACGTGCAATGTTGGCCTA"]
k=8
t=5
n=100
GibbsSampler(Dna, k, t, n)

['GGTGTTCA', 'TAAGTGCC', 'GAAGTATA', 'CAGGTGCA', 'CACGTGCA']

# 1- GreedyMotifSearch_with DosR Example
Best Motifs=['GTTAGGGCCGGAAGT', 'CCGATCGGCATCACT', 'ACCGTCGATGTGCCC', 'GGGTCAGGTATATTT', 'GTGACCGACGTCCCC', 'CTGTTCGCCGGCAGC', 'CTGTTCGATATCACC', 'GTACATGTCCAGAGC', 'GCGATAGGTGAGATT', 'CTCATCGCTGTCATC']
Scores(Motifs): 64

2-GreedyMotifSearchWithPseudocounts_with DosR Example
Best Motifs=['GGACTTCAGGCCCTA', 'GGTCAAACGACCCTA', 'GGACGTAAGTCCCTA', 'GGATTACCGACCGCA', 'GGCCGAACGACCCTA', 'GGACCTTCGGCCCCA', 'GGACTTCTGTCCCTA', 'GGACTTTCGGCCCTG', 'GGACTAACGGCCCTC', 'GGACCGAAGTCCCCG']
Scores(Motifs):35

3- BestRepeatedRandomizedMotifSearch(Dna, k, t, N)_with DosR Example
Best Motifs=['GGACTTCAGGCCCTA', 'GGTCAAACGACCCTA', 'GGACGTAAGTCCCTA', 'GGGCTTCCAACCGTG', 'GGCCGAACGACCCTA', 'GGACCTTCGGCCCCA', 'GGACTTCTGTCCCTA', 'GGACTTTCGGCCCTG', 'GGACTAACGGCCCTC', 'GAGCTATCGGACCTC']
Scores(Motifs): 35

4-GibbsSampler(Dna,k,t,N)_with DosR Example
Best Motifs=['GACGAATGACCCCAG', 'GTCAAACGACCCTAG', 'GGTCATCGACCCCAC', 'GATTACCGACCGCAG', 'GACCGACGTCCCCAG', 'GACCTTCGGCCCCAC', 'GATCAACCTCCGCTG', 'GACTTTCGGCCCTGT', 'GACCAACGCCCCTGG', 'GACCGAAGTCCCCGG']
Scores(Motifs):38

In [12]:
Dna=["GCGCCCCGCCCGGACAGCCATGCGCTAACCCTGGCTTCGATGGCGCCGGCTCAGTTAGGGCCGGAAGTCCCCAATGTGGCAGACCTTTCGCCCCTGGCGGACGAATGACCCCAGTGGCCGGGACTTCAGGCCCTATCGGAGGGCTCCGGCGCGGTGGTCGGATTTGTCTGTGGAGGTTACACCCCAATCGCAAGGATGCATTATGACCAGCGAGCTGAGCCTGGTCGCCACTGGAAAGGGGAGCAACATC", "CCGATCGGCATCACTATCGGTCCTGCGGCCGCCCATAGCGCTATATCCGGCTGGTGAAATCAATTGACAACCTTCGACTTTGAGGTGGCCTACGGCGAGGACAAGCCAGGCAAGCCAGCTGCCTCAACGCGCGCCAGTACGGGTCCATCGACCCGCGGCCCACGGGTCAAACGACCCTAGTGTTCGCTACGACGTGGTCGTACCTTCGGCAGCAGATCAGCAATAGCACCCCGACTCGAGGAGGATCCCG", "ACCGTCGATGTGCCCGGTCGCGCCGCGTCCACCTCGGTCATCGACCCCACGATGAGGACGCCATCGGCCGCGACCAAGCCCCGTGAAACTCTGACGGCGTGCTGGCCGGGCTGCGGCACCTGATCACCTTAGGGCACTTGGGCCACCACAACGGGCCGCCGGTCTCGACAGTGGCCACCACCACACAGGTGACTTCCGGCGGGACGTAAGTCCCTAACGCGTCGTTCCGCACGCGGTTAGCTTTGCTGCC", "GGGTCAGGTATATTTATCGCACACTTGGGCACATGACACACAAGCGCCAGAATCCCGGACCGAACCGAGCACCGTGGGTGGGCAGCCTCCATACAGCGATGACCTGATCGATCATCGGCCAGGGCGCCGGGCTTCCAACCGTGGCCGTCTCAGTACCCAGCCTCATTGACCCTTCGACGCATCCACTGCGCGTAAGTCGGCTCAACCCTTTCAAACCGCTGGATTACCGACCGCAGAAAGGGGGCAGGAC", "GTAGGTCAAACCGGGTGTACATACCCGCTCAATCGCCCAGCACTTCGGGCAGATCACCGGGTTTCCCCGGTATCACCAATACTGCCACCAAACACAGCAGGCGGGAAGGGGCGAAAGTCCCTTATCCGACAATAAAACTTCGCTTGTTCGACGCCCGGTTCACCCGATATGCACGGCGCCCAGCCATTCGTGACCGACGTCCCCAGCCCCAAGGCCGAACGACCCTAGGAGCCACGAGCAATTCACAGCG", "CCGCTGGCGACGCTGTTCGCCGGCAGCGTGCGTGACGACTTCGAGCTGCCCGACTACACCTGGTGACCACCGCCGACGGGCACCTCTCCGCCAGGTAGGCACGGTTTGTCGCCGGCAATGTGACCTTTGGGCGCGGTCTTGAGGACCTTCGGCCCCACCCACGAGGCCGCCGCCGGCCGATCGTATGACGTGCAATGTACGCCATAGGGTGCGTGTTACGGCGATTACCTGAAGGCGGCGGTGGTCCGGA", "GGCCAACTGCACCGCGCTCTTGATGACATCGGTGGTCACCATGGTGTCCGGCATGATCAACCTCCGCTGTTCGATATCACCCCGATCTTTCTGAACGGCGGTTGGCAGACAACAGGGTCAATGGTCCCCAAGTGGATCACCGACGGGCGCGGACAAATGGCCCGCGCTTCGGGGACTTCTGTCCCTAGCCCTGGCCACGATGGGCTGGTCGGATCAAAGGCATCCGTTTCCATCGATTAGGAGGCATCAA", "GTACATGTCCAGAGCGAGCCTCAGCTTCTGCGCAGCGACGGAAACTGCCACACTCAAAGCCTACTGGGCGCACGTGTGGCAACGAGTCGATCCACACGAAATGCCGCCGTTGGGCCGCGGACTAGCCGAATTTTCCGGGTGGTGACACAGCCCACATTTGGCATGGGACTTTCGGCCCTGTCCGCGTCCGTGTCGGCCAGACAAGCTTTGGGCATTGGCCACAATCGGGCCACAATCGAAAGCCGAGCAG", "GGCAGCTGTCGGCAACTGTAAGCCATTTCTGGGACTTTGCTGTGAAAAGCTGGGCGATGGTTGTGGACCTGGACGAGCCACCCGTGCGATAGGTGAGATTCATTCTCGCCCTGACGGGTTGCGTCTGTCATCGGTCGATAAGGACTAACGGCCCTCAGGTGGGGACCAACGCCCCTGGGAGATAGCGGTCCCCGCCAGTAACGTACCGCTGAACCGACGGGATGTATCCGCCCCAGCGAAGGAGACGGCG", "TCAGCACCATGACCGCCTGGCCACCAATCGCCCGTAACAAGCGGGACGTCCGCGACGACGCGTGCGCTAGCGCCGTGGCGGTGACAACGACCAGATATGGTCCGAGCACGCGGGCGAACCTCGTGTTCTGGCCTCGGCCAGTTGTGTAGAGCTCATCGCTGTCATCGAGCGATATCCGACCACTGATCCAAGTCGGGGGCTCTGGGGACCGAAGTCCCCGGGCTCGGAGCTATCGGACCTCACGATCACC"]

# set t equal to the number of strings in Dna, k equal to 15, and N equal to 100
t=len(Dna)
k=15
N=100

# Call GibbsSampler(Dna, k, t, N) 20 times and store the best output in a variable called BestMotifs
BestMotifs=GibbsSampler(Dna, k, t, N)
for i in range(20):
    M=GibbsSampler(Dna, k, t, N)
    if Score(M)<Score(BestMotifs):
        BestMotifs=M
    
    

# Print the BestMotifs variable
print("Best Motifs",BestMotifs)

# Print Score(BestMotifs)
print("score", Score(BestMotifs))

Best Motifs ['GGGACTTCAGGCCCT', 'GGGTCAAACGACCCT', 'GGGACGTAAGTCCCT', 'GGGTGGGCAGCCTCC', 'GGGGCGAAAGTCCCT', 'AGGACCTTCGGCCCC', 'GGGACTTCTGTCCCT', 'GGGACTTTCGGCCCT', 'AGGACTAACGGCCCT', 'GGGACCGAAGTCCCC']
score 37
